# 3_cross_validation_on_classification.ipynb

## Data import and test

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, zero_one_loss
from tqdm.auto import tqdm

In [12]:
name_data_file="heart_failure_clinical_records_dataset"

data = pd.read_csv(f"../../raw_data/{name_data_file}.csv", na_values=["?"])

In [13]:
# Split the data frame into features and labels
X = data.drop(columns=["DEATH_EVENT"])
y = data["DEATH_EVENT"]

N, M = X.shape
print(f"Data loaded: {N} samples, {M} features")

Data loaded: 299 samples, 12 features


# Auxiliary Functions


In [14]:
def get_fold_data(X, y, train_idx, val_idx):
    X_train = X.iloc[train_idx]
    X_val   = X.iloc[val_idx]
    y_train = y.iloc[train_idx]
    y_val   = y.iloc[val_idx]
    return X_train, X_val, y_train, y_val


def get_fold_data_normalized(X, y, train_idx, val_idx):
    X_train, X_val, y_train, y_val = get_fold_data(X, y, train_idx, val_idx)
    mean = X_train.mean(axis=0)
    std  = X_train.std(axis=0)
    X_train_norm = (X_train - mean) / std
    X_val_norm   = (X_val   - mean) / std
    return X_train_norm, X_val_norm, y_train, y_val


def torch_tensor_conversion(X_train, y_train, X_val, y_val):
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32).view(-1, 1)
    X_val_tensor   = torch.tensor(X_val.values, dtype=torch.float32)
    y_val_tensor   = torch.tensor(y_val.values.reshape(-1, 1), dtype=torch.float32).view(-1, 1)
    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor



# Parameters

In [15]:
outer_folds_k_1 = 10
inner_folds_k_2 = 10
random_state = 42

# Hyperparameters to test
hyperparameters_tree = [2, 3, 4, 5, 6, 7]       # e.g. max_depth values
lambdas_logistic = np.logspace(-4, 3, 7) # e.g. inverse of regularization strength C


# Cross_Validation

In [18]:
# Outer CV
CV_outer = StratifiedKFold(n_splits=outer_folds_k_1, shuffle=True, random_state=random_state)

fold_results = {}
best_hyperparameters_logistic = {}
best_hyperparameters_tree = {}
outer_fold_index = 0

for outer_train_idx, outer_test_idx in CV_outer.split(X, y):
    outer_fold_index += 1
    print(f"  OUTER FOLD {outer_fold_index}")

    X_train_outer, X_test_outer = X.iloc[outer_train_idx].copy(), X.iloc[outer_test_idx].copy()
    y_train_outer, y_test_outer = y.iloc[outer_train_idx].copy(), y.iloc[outer_test_idx].copy()

    mu_X_train = X_train_outer.mean(axis=0)
    sigma_X_train = X_train_outer.std(axis=0, ddof=0)
    X_train_outer_scaled = (X_train_outer - mu_X_train) / sigma_X_train
    X_test_outer_scaled  = (X_test_outer  - mu_X_train) / sigma_X_train

    CV_inner = StratifiedKFold(n_splits=inner_folds_k_2, shuffle=True, random_state=outer_fold_index)
    inner_acc_baseline = {}
    inner_acc_logistic = {}
    inner_acc_tree = {}
    inner_fold_index = 0

    for inner_train_idx, inner_test_idx in CV_inner.split(X_train_outer_scaled, y_train_outer):
        inner_fold_index += 1
        print(f"Outer Fold {outer_fold_index} - Inner Fold {inner_fold_index}")

        X_train_inner = X_train_outer_scaled.iloc[inner_train_idx]
        X_val_inner   = X_train_outer_scaled.iloc[inner_test_idx]
        y_train_inner = y_train_outer.iloc[inner_train_idx]
        y_val_inner   = y_train_outer.iloc[inner_test_idx]

        ############################# BASELINE Inner Fold ####################################
        majority_class = y_train_inner.mode()[0]
        y_val_pred = np.full_like(y_val_inner, fill_value=majority_class)
        acc = accuracy_score(y_val_inner, y_val_pred)
        inner_acc_baseline[inner_fold_index] = acc

        ############################# LOGISTIC REGRESSION Inner Fold ####################################
        results_inner_logistic = {lam: {'val_error': []} for lam in lambdas_logistic}

        for lam in lambdas_logistic:
            model = LogisticRegression(penalty="l2", C=1/lam, solver="lbfgs", max_iter=1000, random_state=random_state)
            model.fit(X_train_inner, y_train_inner)
            y_val_pred = model.predict(X_val_inner)
            val_error = zero_one_loss(y_val_inner, y_val_pred)
            results_inner_logistic[lam]['val_error'].append(val_error)

        inner_acc_logistic[inner_fold_index] = results_inner_logistic

        ############################# DECISION TREE Inner Fold ####################################
        results_inner_tree = {depth: {'val_error': []} for depth in hyperparameters_tree}

        for depth in hyperparameters_tree:
            tree = DecisionTreeClassifier(max_depth=depth, criterion='log_loss', random_state=random_state)
            tree.fit(X_train_inner, y_train_inner)
            y_val_pred = tree.predict(X_val_inner)
            val_error = zero_one_loss(y_val_inner, y_val_pred)
            results_inner_tree[depth]['val_error'].append(val_error)

        inner_acc_tree[inner_fold_index] = results_inner_tree

    ############################# LOGISTIC REGRESSION Outer Fold ####################################
    avg_val_error_per_lambda = {}
    for lam in lambdas_logistic:
        val_errors = []
        for inner_fold in inner_acc_logistic.keys():
            val_errors.extend(inner_acc_logistic[inner_fold][lam]['val_error'])
        avg_val_error_per_lambda[lam] = np.mean(val_errors)

    best_lambda = min(avg_val_error_per_lambda, key=avg_val_error_per_lambda.get)
    best_hyperparameters_logistic[outer_fold_index] = best_lambda
    print(f"  ▶ Best λ (regularization): {best_lambda:.5f}")

    final_model = LogisticRegression(penalty="l2", C=1/best_lambda, solver="lbfgs",
                                     max_iter=1000, random_state=random_state)
    final_model.fit(X_train_outer_scaled, y_train_outer)
    y_test_pred = final_model.predict(X_test_outer_scaled)
    outer_test_error = zero_one_loss(y_test_outer, y_test_pred)
    outer_test_acc = accuracy_score(y_test_outer, y_test_pred)

    print(f"  Logistic Regression Outer Test Accuracy: {outer_test_acc:.4f} | Error: {outer_test_error:.4f}")

    ############################# BASELINE Outer Fold ####################################
    mean_acc_inner_baseline = np.mean(list(inner_acc_baseline.values()))
    majority_class_outer = y_train_outer.mode()[0]
    y_pred_baseline_outer = np.full_like(y_test_outer, fill_value=majority_class_outer)
    baseline_acc_outer = accuracy_score(y_test_outer, y_pred_baseline_outer)
    baseline_error_outer = zero_one_loss(y_test_outer, y_pred_baseline_outer)

    ############################# DECISION TREE Outer Fold ####################################
    avg_val_error_per_depth = {
        depth: np.mean([err
                        for fold in inner_acc_tree.keys()
                        for err in inner_acc_tree[fold][depth]['val_error']])
        for depth in hyperparameters_tree
    }
    best_depth = min(avg_val_error_per_depth, key=avg_val_error_per_depth.get)
    best_hyperparameters_tree[outer_fold_index] = best_depth
    print(f"  ▶ Best Tree Depth: {best_depth}")

    final_tree = DecisionTreeClassifier(max_depth=best_depth, criterion='log_loss', random_state=random_state)
    final_tree.fit(X_train_outer_scaled, y_train_outer)
    y_test_pred_tree = final_tree.predict(X_test_outer_scaled)
    outer_test_error_tree = zero_one_loss(y_test_outer, y_test_pred_tree)
    outer_test_acc_tree = accuracy_score(y_test_outer, y_test_pred_tree)

    ############################# STORE RESULTS ####################################
    fold_results[outer_fold_index] = {
        "baseline_mean_inner_acc": mean_acc_inner_baseline,
        "baseline_outer_acc": baseline_acc_outer,
        "baseline_outer_error": baseline_error_outer,
        "logistic_best_lambda": best_lambda,
        "logistic_outer_acc": outer_test_acc,
        "logistic_outer_error": outer_test_error,
        "tree_best_depth": best_depth,
        "tree_outer_acc": outer_test_acc_tree,
        "tree_outer_error": outer_test_error_tree
    }

    print(f"Finished Outer Fold {outer_fold_index}")

print(" Nested CV Completed ")
print("Best Logistic Regression λ per fold:")
print(best_hyperparameters_logistic)
print("Best Tree Depth per fold:")
print(best_hyperparameters_tree)

outer_results_df = pd.DataFrame.from_dict(fold_results, orient='index')
print("\n=== Summary of Outer Fold Results ===")
print(outer_results_df)

  OUTER FOLD 1
Outer Fold 1 - Inner Fold 1
Outer Fold 1 - Inner Fold 2
Outer Fold 1 - Inner Fold 3
Outer Fold 1 - Inner Fold 4
Outer Fold 1 - Inner Fold 5
Outer Fold 1 - Inner Fold 6
Outer Fold 1 - Inner Fold 7
Outer Fold 1 - Inner Fold 8
Outer Fold 1 - Inner Fold 9
Outer Fold 1 - Inner Fold 10
  ▶ Best λ (regularization): 4.64159
  Logistic Regression Outer Test Accuracy: 0.7667 | Error: 0.2333
  ▶ Best Tree Depth: 5
Finished Outer Fold 1
  OUTER FOLD 2
Outer Fold 2 - Inner Fold 1
Outer Fold 2 - Inner Fold 2
Outer Fold 2 - Inner Fold 3
Outer Fold 2 - Inner Fold 4
Outer Fold 2 - Inner Fold 5
Outer Fold 2 - Inner Fold 6
Outer Fold 2 - Inner Fold 7
Outer Fold 2 - Inner Fold 8
Outer Fold 2 - Inner Fold 9
Outer Fold 2 - Inner Fold 10
  ▶ Best λ (regularization): 4.64159
  Logistic Regression Outer Test Accuracy: 0.8667 | Error: 0.1333
  ▶ Best Tree Depth: 2
Finished Outer Fold 2
  OUTER FOLD 3
Outer Fold 3 - Inner Fold 1
Outer Fold 3 - Inner Fold 2
Outer Fold 3 - Inner Fold 3
Outer Fold 3 

In [ ]:
CV_outer = StratifiedKFold(n_splits=outer_folds_k_1, shuffle=True, random_state=random_state)

fold_results = {}
best_hyperparameters_logistic = {}
best_hyperparameters_tree = {}
outer_fold_index = 0

for outer_train_idx, outer_test_idx in CV_outer.split(X, y):
    outer_fold_index += 1
    print(f"  OUTER FOLD {outer_fold_index}")

    ############################ DATA Outer Fold ####################################

    X_train_outer, X_test_outer, y_train_outer, y_test_outer = get_fold_data(X, y, outer_train_idx, outer_test_idx)

    CV_inner = StratifiedKFold(n_splits=inner_folds_k_2, shuffle=True, random_state=outer_fold_index)
    inner_acc_baseline = {}
    inner_acc_logistic = {}
    inner_acc_tree = {}
    inner_fold_index = 0

    for inner_train_idx, inner_test_idx in CV_inner.split(X_train_outer, y_train_outer):
        inner_fold_index += 1
        print(f"Outer Fold {outer_fold_index} - Inner Fold {inner_fold_index}")

        ############################ DATA Inner Fold ####################################

        X_train_inner_norm, X_test_inner_norm, y_train_inner_norm, y_test_inner_norm = get_fold_data_normalized(X_train_outer, y_train_outer, inner_train_idx, inner_test_idx)

        ############################# LOGISTIC REGRESSION Inner Fold ####################################
        results_inner_logistic = {lam: {'val_error': []} for lam in lambdas_logistic}

        for lam in lambdas_logistic:
            model = LogisticRegression(penalty="l2", C=1/lam, solver="lbfgs", max_iter=1000, random_state=random_state)
            model.fit(X_train_inner_norm, y_train_inner_norm)
            y_val_pred = model.predict(X_test_inner_norm)
            val_error = zero_one_loss(y_test_inner_norm, y_val_pred)
            results_inner_logistic[lam]['val_error'].append(val_error)

        inner_acc_logistic[inner_fold_index] = results_inner_logistic

        ############################# DECISION TREE Inner Fold ####################################

        results_inner_tree = {depth: {'val_error': []} for depth in hyperparameters_tree}

        for depth in hyperparameters_tree:
            tree = DecisionTreeClassifier(max_depth=depth, criterion='log_loss', random_state=random_state)
            tree.fit(X_train_inner_norm, y_train_inner_norm)
            y_val_pred = tree.predict(X_test_inner_norm)
            val_error = zero_one_loss(y_test_inner_norm, y_val_pred)
            results_inner_tree[depth]['val_error'].append(val_error)

        inner_acc_tree[inner_fold_index] = results_inner_tree

        ############################# BASELINE Inner Fold (REMOVED) ####################################

        # -----

        ######################## OUTER FOLD ##########################################################

    ############################ Data ####################################

    X_train_outer_norm, X_test_outer_norm, y_train_outer_norm, y_test_outer_norm = get_fold_data_normalized(X, y, outer_train_idx, outer_test_idx)

    ############################# LOGISTIC REGRESSION Outer Fold ####################################

    avg_val_error_per_lambda = {}
    for lam in lambdas_logistic:
        val_errors = []
        for inner_fold in inner_acc_logistic.keys():
            val_errors.extend(inner_acc_logistic[inner_fold][lam]['val_error'])
        avg_val_error_per_lambda[lam] = np.mean(val_errors)

    best_lambda = min(avg_val_error_per_lambda, key=avg_val_error_per_lambda.get)
    best_hyperparameters_logistic[outer_fold_index] = best_lambda
    print(f"  ▶ Best λ (regularization): {best_lambda:.5f}")

    final_model = LogisticRegression(
        penalty="l2", C=1/best_lambda, solver="lbfgs", max_iter=1000, random_state=random_state
    )
    final_model.fit(X_train_outer_norm, y_train_outer_norm)
    y_test_pred = final_model.predict(X_test_outer_norm)
    outer_test_error = zero_one_loss(y_test_outer_norm, y_test_pred)
    outer_test_acc = accuracy_score(y_test_outer_norm, y_test_pred)

    print(f"  Logistic Regression Outer Test Accuracy: {outer_test_acc:.4f} | Error: {outer_test_error:.4f}")

    ############################# DECISION TREE Outer Fold ####################################

    avg_val_error_per_depth = {
        depth: np.mean([
            err for fold in inner_acc_tree.keys()
            for err in inner_acc_tree[fold][depth]['val_error']
        ])
        for depth in hyperparameters_tree
    }

    best_depth = min(avg_val_error_per_depth, key=avg_val_error_per_depth.get)
    best_hyperparameters_tree[outer_fold_index] = best_depth
    print(f"  ▶ Best Tree Depth: {best_depth}")

    final_tree = DecisionTreeClassifier(max_depth=best_depth, criterion='log_loss', random_state=random_state)
    final_tree.fit(X_train_outer_norm, y_train_outer_norm)
    y_test_pred_tree = final_tree.predict(X_test_outer_norm)
    outer_test_error_tree = zero_one_loss(y_test_outer_norm, y_test_pred_tree)
    outer_test_acc_tree = accuracy_score(y_test_outer_norm, y_test_pred_tree)

    ############################# BASELINE Outer Fold ####################################

    majority_class_outer = y_train_outer_norm.mode()[0]
    y_pred_baseline_outer = np.full_like(y_test_outer_norm, fill_value=majority_class_outer)
    baseline_acc_outer = accuracy_score(y_test_outer_norm, y_pred_baseline_outer)
    baseline_error_outer = zero_one_loss(y_test_outer_norm, y_pred_baseline_outer)


    ############################# STORE RESULTS ####################################
    fold_results[outer_fold_index] = {
        "baseline_outer_acc": baseline_acc_outer,
        "baseline_outer_error": baseline_error_outer,
        "logistic_best_lambda": best_lambda,
        "logistic_outer_acc": outer_test_acc,
        "logistic_outer_error": outer_test_error,
        "tree_best_depth": best_depth,
        "tree_outer_acc": outer_test_acc_tree,
        "tree_outer_error": outer_test_error_tree
    }

    print(f"Finished Outer Fold {outer_fold_index}")

print("Nested CV Completed")
print("Best Logistic Regression λ per fold:")
print(best_hyperparameters_logistic)
print("Best Tree Depth per fold:")
print(best_hyperparameters_tree)

outer_results_df = pd.DataFrame.from_dict(fold_results, orient='index')
print("\n=== Summary of Outer Fold Results ===")
print(outer_results_df)


  OUTER FOLD 1
Outer Fold 1 - Inner Fold 1
Outer Fold 1 - Inner Fold 2
Outer Fold 1 - Inner Fold 3
Outer Fold 1 - Inner Fold 4
Outer Fold 1 - Inner Fold 5
Outer Fold 1 - Inner Fold 6
Outer Fold 1 - Inner Fold 7
Outer Fold 1 - Inner Fold 8
Outer Fold 1 - Inner Fold 9
Outer Fold 1 - Inner Fold 10
  ▶ Best λ (regularization): 4.64159
  Logistic Regression Outer Test Accuracy: 0.7667 | Error: 0.2333
  ▶ Best Tree Depth: 5
Finished Outer Fold 1
  OUTER FOLD 2
Outer Fold 2 - Inner Fold 1
Outer Fold 2 - Inner Fold 2
Outer Fold 2 - Inner Fold 3
Outer Fold 2 - Inner Fold 4
Outer Fold 2 - Inner Fold 5
Outer Fold 2 - Inner Fold 6
Outer Fold 2 - Inner Fold 7
Outer Fold 2 - Inner Fold 8
Outer Fold 2 - Inner Fold 9
Outer Fold 2 - Inner Fold 10
  ▶ Best λ (regularization): 4.64159
  Logistic Regression Outer Test Accuracy: 0.8667 | Error: 0.1333
  ▶ Best Tree Depth: 2
Finished Outer Fold 2
  OUTER FOLD 3
Outer Fold 3 - Inner Fold 1
Outer Fold 3 - Inner Fold 2
Outer Fold 3 - Inner Fold 3
Outer Fold 3 

code


In [ ]:
m = 10 # Repetitions
K = 10 # Folds
rho = 1 / K # Correlation heuristic
alpha = 0.05 # Significance level

# ANN parameters

input_dim  = M # M number of features
output_dim = 1 # regression problem
lr = 1e-3
n_epochs = 1000
momentum = 0.9

#Loss Function 
l2_loss = lambda y, y_pred: (y - y_pred)**2
loss_func = l2_loss # Loss function

# Parameters used

best_lambda_statistic_test = best_lambda
best_hyperparameter_statistic_test = best_hyperparameter

In [ ]:
r = []

for repeat_idx in range(m):
    print(f"Repetition {repeat_idx+1}/{m}")

    # 5.2) Initialize KFold cross-validation, set the seed to repeat_idx
    ### BEGIN SOLUTION
    CV_kfold = KFold(n_splits=K, shuffle=True, random_state=repeat_idx)
    ### END SOLUTION

    for fold, (train_index, test_index) in tqdm(enumerate(CV_kfold.split(X)), total=CV_kfold.get_n_splits(X),desc="Cross-validation fold"):
        # Split data into training and test sets

        ############################################# DATA #################################################

        X_train_norm, X_test_norm, y_train_norm, y_test_norm= get_fold_data_normalized(X, y, train_index, test_index)

        X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor = torch_tensor_conversion(X_train_norm, y_train_norm, X_test_norm, y_test_norm)

        ############################################# LINEAR REGRESSION #################################################

        model = LogisticRegression(penalty="l2", C=1/lam, solver="lbfgs", max_iter=1000, random_state=random_state)
        model.fit(X_train_norm, y_train_norm)

        y_test_linear_reg = model.predict(X_test_norm)
        loss_func_linear_reg = loss_func(y_test_norm, y_test_linear_reg)

        ############################################# BASELINE #################################################

        y_train_mean_baseline = y_train_norm.mean()
        y_test_pred_baseline = pd.Series(y_train_mean_baseline, index=y_test_norm.index)
        loss_funcion_baseline = loss_func(y_test_norm, y_test_pred_baseline)


        ######################################################### MODELS COMPARISON #######################################
        
        r_j = np.mean(loss_func_linear_reg - loss_funcion_baseline)

        r.append(r_j)

# Calculate p-value and confidence interval using correlated t-test
r_hat, CI, p_value = correlated_ttest(r, rho, alpha=alpha)

print(f"\nSetup II results:")
print(f"r_hat: {r_hat:.4f}")
print(f"95% CI: [{CI[0]:.4f}, {CI[1]:.4f}]")
print(f"p-value: {p_value}")